In [1]:
%pwd

'e:\\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS'

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import sentence_transformers

import langchain

e:\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS\llmbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install -q langchain

print(f'Langchain Version: {langchain.__version__}')
import sys
print(f'Python Version: {sys.version}')

Langchain Version: 1.2.8
Python Version: 3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]


In [6]:
# Extract the Data From PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob= "*pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data= load_pdf_file(data='Data/')

Ignoring wrong pointing object 0 0 (offset 0)


In [8]:
# extracted_data

In [9]:
# Split the Data into the Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks", len(text_chunks))

Length of Text chunks 7683


In [11]:
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2") 

In [12]:

embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2") 


C:\Users\ragha\AppData\Local\Temp\ipykernel_7916\2622195746.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")


In [13]:
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))

Length 1024


In [14]:
# query_result

# VectorDB(PINECONE) Connection

In [15]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY = api_key=os.environ.get("PINECONE_API_KEY")

In [17]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "llm-med-bot"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY   

In [19]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Embed each chunk and upsert the  embeddings into your Pinecone index

from langchain_pinecone.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)
